## LightGBM 分類モデルの SHAP による説明と Error Analysis + Azure ML
LightGBM で構築した年収予測 (50kドル以上か以下かを予測する分類問題)のモデルを Interpret-community の SHAPベースの explainer を用いて説明 (グローバル、ローカル) を行います。また  [Error Analysis](https://erroranalysis.ai/) を用いてモデルの誤差が大きいコホートを抽出します。

### 0. 事前準備
- Jupyter Kernel :  `rai-aml` を選択する。
    - [0-Setup.ipynb](./0-Setup.ipynb) の手順に従い構築しておくこと。


- Azure ML Dataset : `adult_census` を利用するため登録済みか確認する。
    - [0-Setup.ipynb](./0-Setup.ipynb) の手順に従い登録しておくこと。

### 1. ライブラリ
必要な Python ライブラリをインポートします。

In [ ]:
from azureml.core import Run, Experiment, Workspace, Model, Dataset
from azureml.interpret import ExplanationClient

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score

from lightgbm import LGBMClassifier

from raiwidgets import ExplanationDashboard

import joblib
import os

In [ ]:
# interpret-community 関連
from interpret.ext.blackbox import TabularExplainer
from interpret_community.common.constants import ModelTask

### 2. Azure ML Workspace 設定
Azure Machine Learning Workspace と接続します。

In [ ]:
ws = Workspace.from_config()

### 3. データ準備
adult census の Dataset を呼び出し Pandas DataFrame に変換します。

In [ ]:
df = Dataset.get_by_name(ws, name='adult_census').to_pandas_dataframe().sample(1000, random_state=1234)
df.head()

In [ ]:
# 簡単のため race=Other のデータを除去する
df_raw = df[df.race!='Other']

In [ ]:
# 説明変数と目的変数に分離
X = df_raw.drop(['target'], axis=1)
Y = df_raw['target']
Y = (Y == '>50K') * 1

In [ ]:
# 学習データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, Y, 
                                                    test_size=0.3, 
                                                    random_state=1234)

### 4. 学習パイプライン

#### Azure ML 実験の設定

In [ ]:
exp = Experiment(ws, "rai-explain-erroranalysis")
run = exp.start_logging(snapshot_directory=None)

#### LightGBM パラメータ設定

In [ ]:
## LightGBM Parameter list (https://lightgbm.readthedocs.io/en/latest/Parameters.html)
params = {
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'n_estimators':1000,
    'metric': 'auc',
    'colsample_bytree': 1.0,
    'reg_alpha': 1e-3,
    'reg_lambda': 1e-3,
    'seed': 1234,
}

In [ ]:
params_fit = {
              "classifier__verbose":10, 
             }

#### scikit learn pipeline 構築

In [ ]:
features = X.columns.values.tolist()
classes = ['<=50K' ,'>50K']
feat_list = {
'num_cols': X.dtypes[X.dtypes == 'float64'].index.values.tolist(),
'cat_cols': X.dtypes[X.dtypes == 'object'].index.values.tolist(),
}
print(feat_list)

num_pipe = Pipeline([
    ('num_imputer', SimpleImputer(strategy='median')),
    ('num_scaler', StandardScaler())
])
cat_pipe = Pipeline([
    ('cat_imputer', SimpleImputer(strategy='constant', fill_value='?')),
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
])
feat_pipe = ColumnTransformer([
    ('num_pipe', num_pipe, feat_list['num_cols']),
    ('cat_pipe', cat_pipe, feat_list['cat_cols'])
])


model = Pipeline(
    steps=[
        ("preprocessor", feat_pipe),
        (
            "classifier",
            LGBMClassifier(**params),
        ),
    ]
)

### 5. モデル学習

In [ ]:
# scikit learn パイプラインの実行
model.fit(X_train, y_train, **params_fit);

モデルの精度を確認し、メトリックとして記録します。

In [ ]:
run.log('train AUC',roc_auc_score(y_train, model.predict(X_train)))

In [ ]:
run.log('test AUC',roc_auc_score(y_test, model.predict(X_test)))

In [ ]:
run.get_metrics()

In [ ]:
run

### 6. モデル説明 (SHAP Tabular Explainer)
interpret-community の `TabularExplainer` を用いて explainer を生成します。TabularExplainer は最適な [SHAP Explainer](https://github.com/interpretml/interpret-community#supported-explainers) を自動で選択します。

In [ ]:
explainer = TabularExplainer(model.steps[-1][1],
                             initialization_examples=X_train, # データの母集団を引数に渡す。テストデータ X_test でも可。
                             features=features,
                             classes=classes,
                             transformations=feat_pipe,
                             model_task = ModelTask.Classification)

In [ ]:
global_explanation = explainer.explain_global(X_train)
local_explanation = explainer.explain_local(X_test)

In [ ]:
client = ExplanationClient.from_run(run)
client.upload_model_explanation(global_explanation, 
                                comment='global explanation', 
                                true_ys=y_train.values)

In [ ]:
from raiwidgets import ExplanationDashboard
ExplanationDashboard(global_explanation, model, dataset=X_train, true_y=y_train.values)

In [ ]:
# テストデータ
client.upload_model_explanation(local_explanation, 
                                comment='local explanation',
                                true_ys=y_test.values
                               )
ExplanationDashboard(local_explanation, model, dataset=X_test, true_y=y_test)

### 7. モデル誤差分析 (Error Analysis)
[Error Analysis](https://erroranalysis.ai/) を用いてモデルの誤差を分析し、特に誤差が大きいコホートを特定します。

In [ ]:
from raiwidgets import ErrorAnalysisDashboard

In [ ]:
# ErrorAnalysisDashboard(global_explanation, model,
#                        dataset=X_train, 
#                        true_y=y_train.values, 
#                        categorical_features=feat_list['cat_cols'],
#                       )

In [ ]:
ErrorAnalysisDashboard(local_explanation, model,
                       dataset=X_test, 
                       true_y=y_test.to_numpy(),
                       model_task="classification",
                       categorical_features=feat_list['cat_cols'],
                       true_y_dataset=y_test.to_numpy())

In [ ]:
# 実験環境
run.complete()

In [ ]:
run